# Inverters

In [1]:
from z3 import *
import numpy as np

## Definição do sistema e da sua evolução temporal

Vamos representar o nosso problema com um FOTS $\; \Sigma \equiv \langle \mathcal{T},\mathcal{M},\mathbf{init},\mathbf{trans}\rangle\;$. Seja $\mathcal{I}$ o nosso conjunto de inversores.

 - $\mathcal{T}$ é uma SMT de inteiros

 - O conjunto $\mathcal{M} = \{ (\mathtt{in}_i, \mathtt{out}_i) \; | \; i \in \mathcal{I} \}$ representa o conjunto de entradas e saidas de cada inversor.
 
 - De forma a facilitar a notação, considere-se $\rho_i$ o inversor a seguir ao inversor $i$
 
A função $\mathbf{init}$ pode ser descrita da seguinte forma:

$$\mathbf{init}(m) \equiv \bigwedge_{i \in \mathcal{I}} \Big( \big( \mathtt{in}_i = 0 \vee \mathtt{in}_i = 1 \big)\wedge \big(\mathtt{out}_i = 0 \vee \mathtt{out}_i = 1 \big) \wedge \mathtt{in}_{\rho_i} = \mathtt{out}_i \Big)$$

In [2]:
def declare(i, num_inv=4):
    state = {j: {"in": Int(f"in_s{i}_i{j}"), "out": Int(f"out_s{i}_i{j}")} for j in range(num_inv)}
    return state

def init(state):
    conds = []
    for i in state:
        conds.append(Or(state[i]["in"] == 0, state[i]["in"] == 1))
        conds.append(Or(state[i]["out"] == 0, state[i]["out"] == 1))
        conds.append(state[i]["out"] == state[(i+1)%len(state)]["in"])
    
    return And(conds)

A função $\mathbf{invert}$ define a evolução do output de um inversor, e tem a seguinte definição:

$$\mathbf{invert}(m,m',i) \, \equiv \, \mathtt{out}_i'=\mathtt{out}_i \, \vee \, \mathtt{out}_i'= 1 - \mathtt{in}_i$$

Utilizando esta definição, podemos agora definir a função $\mathbf{trans}(m,m')$:

$$\mathbf{trans}(m,m') \, \equiv \, \bigwedge_{i \in \mathcal{I}} \big( \mathtt{in}_{\rho_i}' = \mathtt{out}_i' \wedge \mathbf{invert}(m,m',i) \big) $$

In [3]:
def invert(inv_atual, inv_ant):
    c1 = inv_atual["out"] == inv_ant["out"]
    c2 = inv_atual["out"] == 1 - inv_ant["in"]
    r = Or(c1, c2)
    
    return r  
    
def trans(sys_atual, sys_ant):
    # Restrições da determinação do output de cada inversor
    conds = []
    for i in sys_atual:
        conds.append(sys_atual[(i+1)%len(sys_atual)]["in"] == sys_atual[i]["out"])
        conds.append(invert(sys_atual[i], sys_ant[i]))
    
    r = And(conds)
    
    return r

Para visualizar a evolução temporal do sistema, foi criada a função $\mathbf{time\_evolution}$, que gera um traco $\alpha$ de $k$ elementos do nosso sistema, definida da seguinte forma:

$$ \mathbf{time\_evolution}(k) \equiv \mathbf{init}(\alpha_0) \wedge \bigwedge_{0\le i \le k-1}\mathbf{trans}(\alpha_{i+1},\alpha_i) $$

Sendo $\alpha_i$ o $i$-ésimo elemento do traco.

In [4]:
def time_evolution(declare, init, trans, k):
    # Criar o solver e o traço de estados temporais
    solver = Solver()
    states = {i: declare(i) for i in range(k)}

    # Inicializar o primeiro estado temporal
    solver.add(init(states[0]))

    # Fazer a transição de estados
    for i in range(1, k):
        solver.add(trans(states[i], states[i-1]))

    if solver.check() == sat:
        m = solver.model()
        
        output = {}
        for i in range(k):
            elems = [elem for elem in m if f"_s{i}_" in str(elem)]
            output[i] = {str(elem): m[elem] for elem in elems if "out" in str(elem)}
        
        for k in output.keys():
            keys = sorted(output[k].keys())
            output[k] = [output[k][key] for key in keys]
            print(k, output[k])
            
    return

time_evolution(declare, init, trans, 10)

0 [0, 1, 1, 1]
1 [0, 1, 1, 1]
2 [0, 1, 1, 0]
3 [1, 1, 1, 0]
4 [1, 1, 1, 0]
5 [1, 0, 0, 0]
6 [1, 0, 0, 0]
7 [1, 0, 0, 1]
8 [0, 0, 0, 1]
9 [0, 1, 0, 1]


### Provar que o programa não termina

Para esta prova é utilizada $k$-indução e $k$-lookahead.

In [5]:
def kinduction_always(declare, init, trans, inv, k, prop="variant decrease"):
    solver = Solver()
    trace = {i: declare(i) for i in range(k+1)}
    solver.add(init(trace[0]))
    
    # Testar k casos de base
    for i in range(k):
        solver.add(trans(trace[i+1], trace[i]))
    solver.add(Or([Not(inv(trace[i])) for i in range(k)]))
    
    # Impedir que estes terminem
    for i in range(k-1):
        solver.add(Not(Sum([trace[i][j]["out"] for j in trace[i]])==0))
        
    if solver.check() == sat:
        print(f"The {prop} breaks down in (at least) one of the initial states.\n")
        m = solver.model()
        
        output = {}
        for i in range(k):
            elems = [elem for elem in m if f"_s{i}_" in str(elem)]
            output[i] = {str(elem): m[elem] for elem in elems if "out" in str(elem)}
        
        for k in output.keys():
            keys = sorted(output[k].keys())
            output[k] = [output[k][key] for key in keys]
            print(k, output[k])
                
        return
    elif solver.check() != unsat:
        return
        
    # Testar caso indutivo
    solver = Solver()
    solver.add(init(trace[0]))
    for i in range(k):
        solver.add(trans(trace[i+1], trace[i]))
        solver.add(inv(trace[i]))
    for i in range(k-1):
        solver.add(Not(Sum([trace[i][j]["out"] for j in trace[i]])==0))    
    solver.add(Not(inv(trace[k])))
    
    if solver.check() == sat:
        print(f"The {prop} fails in the inductive state.\n")
        
        m = solver.model()
        
        output = {}
        for i in range(k):
            elems = [elem for elem in m if f"_s{i}_" in str(elem)]
            output[i] = {str(elem): m[elem] for elem in elems if "out" in str(elem)}
        
        for k in output.keys():
            keys = sorted(output[k].keys())
            output[k] = [output[k][key] for key in keys]
            print(k, output[k])
                
        return
    else:
        print(f"The {prop} holds.")
        
        
def klookahead(declare, init, trans, inv, k, l=2, prop="variant decrease"):
    solver = Solver()
    trace = {i: declare(i) for i in range(k+1)}
    solver.add(init(trace[0]))
    
    # Testar k casos de base
    for i in range(k):
        solver.add(trans(trace[i+1], trace[i]))
    for i in range(k):
        solver.add(Not(Sum([trace[i][j]["out"] for j in trace[i]])==0))
    solver.add(Or([Not(inv(trace[i], l)) for i in range(k)]))
        
    if solver.check() == sat:
        print(f"The {prop} breaks down in (at least) one of the initial states.\n")       
        return
    elif solver.check() != unsat:
        return
        
    # Testar caso indutivo
    solver = Solver()
    solver.add(init(trace[0]))
    for i in range(k):
        solver.add(trans(trace[i+1], trace[i]))
        solver.add(inv(trace[i], l))
    for i in range(k-1):
        solver.add(Not(Sum([trace[i][j]["out"] for j in trace[i]])==0))
    solver.add(Not(inv(trace[k], l)))
    
    if solver.check() == sat:
        print(f"The {prop} fails in the inductive state.\n")
        return
    else:
        print(f"The {prop} holds.")

É, então, definido um variante $V$, definido como a soma dos outputs de todos os inversores.

$$ V \equiv \sum_{i \in\mathcal{I}} \mathtt{out}_i $$

Para provar que este programa não termina é necessário provar que o variante é sempre positivo, que decresce e que é util:

- Positivo: Provar por $k$-indução que, para todos os elementos do traço 

$$ \, \sum_{i \in\mathcal{I}} \mathtt{out}_i \geq 0 $$

- Descrescente: Provar por $k$-lookahead que, para dois elementos $i$ e $i+k$ do traço

$$ V_{i+k} < V_{i} $$

- Util: Provar por $k$-indução que, para todos os elementos do traço

$$ V=0 \rightarrow  \bigwedge_{i\in \mathcal{I}}\mathtt{out}_i = 0  $$

In [6]:
def variant(state):
    return Sum([state[i]["out"] for i in state])

def var_positive(state):
    return variant(state) >= 0

def var_decreases(state, l=2):
    states = {i: declare(-i) for i in range(1, l+1)}
    
    conds = [trans(states[1], state)]
    for i in range(1, l):
        conds += [trans(states[i+1], states[i])]
        
    c1 = And(conds)
    c2 = Or(variant(states[l])<variant(state), variant(states[l])==0)
    
    lista = []
    for i in state:
        lista.append(state[i]["out"])
        lista.append(state[i]["in"])
    
    r = ForAll(lista, Implies(c1, c2))  
    return r

def var_useful(state):
    return Implies(variant(state)==0, And([state[i]["out"]==0 for i in state]))

kinduction_always(declare, init, trans, var_positive, 1, "positivity of the variant")
kinduction_always(declare, init, trans, var_useful, 1, "usefulness of the variant")

The positivity of the variant holds.
The usefulness of the variant holds.


In [37]:
for l in range(1, 20):
    klookahead(declare, init, trans, var_decreases, 1, l)

The variant decrease breaks down in (at least) one of the initial states.

The variant decrease breaks down in (at least) one of the initial states.

The variant decrease breaks down in (at least) one of the initial states.

The variant decrease breaks down in (at least) one of the initial states.

The variant decrease breaks down in (at least) one of the initial states.

The variant decrease breaks down in (at least) one of the initial states.

The variant decrease breaks down in (at least) one of the initial states.

The variant decrease breaks down in (at least) one of the initial states.

The variant decrease breaks down in (at least) one of the initial states.

The variant decrease breaks down in (at least) one of the initial states.

The variant decrease breaks down in (at least) one of the initial states.

The variant decrease breaks down in (at least) one of the initial states.

The variant decrease breaks down in (at least) one of the initial states.

The variant decrease brea

### Provar os estados iniciais para os quais o programa termina

De forma a determinar os estados iniciais que permitem o término do programa, foi criada a função $\mathbf{possible\_states}$, definida da seguinte forma:

$$ \mathbf{possible\_states}(k) \equiv \mathbf{init}(\alpha_0) \wedge \bigwedge_{0\le i \le k-1}\mathbf{trans}(\alpha_{i+1},\alpha_i) \wedge \bigvee_{0\le i \le k} \mathbf{variant}(\alpha_i)=0 $$


In [38]:
def possible_states(declare, init, trans, k):
    solver = Solver()
    trace = {i: declare(i) for i in range(k)}
    solver.add(init(trace[0]))
    
    for i in range(k-1):
        solver.add(trans(trace[i+1], trace[i]))
    solver.add(Or([variant(trace[i])==0 for i in range(k)]))
        
    while solver.check() == sat:
        m = solver.model()
        
        outputs, inputs = [], []
        for i in trace[0]:
            outputs.append([m[elem] for elem in m if f"out_s0_i{i}" in str(elem)][0])
            inputs.append([m[elem] for elem in m if f"in_s0_i{i}" in str(elem)][0])
        
        print(f"Possible starting state: {outputs}")
            
        for i in trace[0]:
            solver.add(trace[0][i]["out"] != outputs[i])
            solver.add(trace[0][i]["in"] != inputs[i])
                
    return
    
possible_states(declare, init, trans, 100)

Possible starting state: [0, 0, 0, 0]
Possible starting state: [1, 1, 1, 1]
